# <center>Function Minimization</center>
## <center>Starting With Empty Model</center>
### <center>Dr David Race</center>

This notebook is specifically designed to use the Deep Learning toolkit (pytorch) to determine the minimum of a non-negative convex function.  The input will be a "loss" function and a starting point, then it finds the location and value of the minimum of the function.

## Set Up Environment

This section installs the basic pytorch capabilities into the Colaboratory VM.

In [0]:
from os import path
!pip3 --version
!pip3 install -U torch torchvision
accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

In [0]:
import numpy as np
import torch
from pprint import pprint, pformat
pprint(accelerator)

## Example 1 - $x^2 + y^2 + 4$

In this example, we will compute the minimum of $x^2 + y + 4^2$ starting at a random point.  The point will be initialized using randn.

In [0]:
#define setup
offset = torch.Tensor([4.], device = device).double()
mean_val = 10.
std_val = 20.
iter_stop = 1e-7
max_iter = 100
def compute_loss_error(l1, l2):
  return np.abs(l1 - l2)/np.max([l1,l2])

def loss_f(in_pos):
  y = in_pos.pow(2).sum() + offset
  return y
  
device = torch.device(accelerator)
np.random.seed(0)
N = 2
x1 = torch.Tensor(std_val * (np.random.randn(1,N) + mean_val),device=device).double()
x1.requires_grad = True
print("x1")
print(x1.data.numpy()[0])
print("grad of x1")
print([2.0 * x1.data.numpy()[0][0], 2.0 * x1.data.numpy()[0][1]])
y = loss_f(x1)
#Optimize Using Gradient Descent
learning_rate = .1
i = 1
last_loss = loss_f(x1).data.numpy()[0]
adj_error = last_loss
while i<=max_iter and adj_error > iter_stop:
  loss = loss_f(x1)
  loss.backward()
  if(i == 1):
    print("Computed Gradient")
    print(x1.grad.numpy()[0])
  with torch.no_grad():
    x1 -= learning_rate * x1.grad
    x1.grad.zero_()
  if i % 10 == 0:  
    curr_loss = loss_f(x1).data.numpy()[0]
    adj_error = compute_loss_error(last_loss,curr_loss)
    last_loss = curr_loss
  i = i + 1
    
print("Iteration " + str(i))
print("Approx location")
print(x1.data.numpy()[0])
pprint("Approx Value")
pprint(loss_f(x1).data.numpy()[0])

You should note that the DL computation for the gradient is the same as the mathematical computation for the gradient.  This gradient is used throughout DL for training, so it is important to know that it is the same as the mathematical expectations.

## 2 - Example 1 using Stochastic Gradient Descent

This method provides the same steps as the previous example, but the application of the gradient descent is carried out automatically by torch.optim.SGD().step().  The torch environment manages the application of the loss to the tensors to change within its framework (this is very handy for processing concepts.)

In [0]:
#define setup
offset = torch.Tensor([4.]).double()
mean_val = 10.
std_val = 20.
iter_stop = 1e-7
max_iter = 1000
def compute_loss_error(l1, l2):
  return np.abs(l1 - l2)/np.max([l1,l2])

def loss_f(in_pos):
  y = in_pos.pow(2).sum() + offset
  return y
  
device = torch.device(accelerator)
N = 2
#reset the seed so the results are consistent
#
np.random.seed(0)
x1 = torch.Tensor(std_val * (np.random.randn(1,N) + mean_val),device=device).double()
x1.requires_grad = True
y = loss_f(x1)
#define optimizer
optimizer = torch.optim.SGD([x1], lr=.1)
#Optimize
i = 1
last_loss = loss_f(x1).data.numpy()[0]
adj_error = last_loss
while i<=max_iter and adj_error > iter_stop:
  optimizer.zero_grad()
  loss = loss_f(x1)
  loss.backward()
  optimizer.step()
  if i % 10 == 0:  
    curr_loss = loss_f(x1).data.numpy()[0]
    adj_error = compute_loss_error(last_loss,curr_loss)
    last_loss = curr_loss
  i = i + 1
    
print("Iteration " + str(i))
print("Approx location")
print(x1.data.numpy()[0])
pprint("Approx Value")
pprint(loss_f(x1).data.numpy()[0])
print('max relative difference')
pprint(np.abs(x1.data.numpy()[0][0] - x1.data.numpy()[0][1])/np.min(np.abs(x1.data.numpy()[0])))

Happily, the results turn out exactly the same.

## Example 3 - Example 1 using Adam Optimizer

The Adam optimizer chooses a slightly different application of the step size based upon the momentum method.  This method is very useful in machine learning, but can be applied in our simple examples using larger learning rates.  This is performed in the following compute cell.  <i>(Normally there is some experimentation to find the correct learning rate, but this notebook starts using 4.0.)</i>

In [0]:
#define setup
offset = torch.Tensor([4.]).double()
mean_val = 10.
std_val = 20.
iter_stop = 1e-7
max_iter = 1000
def compute_loss_error(l1, l2):
  return np.abs(l1 - l2)/np.max([l1,l2])

def loss_f(in_pos):
  y = in_pos.pow(2).sum() + offset
  return y
  
device = torch.device(accelerator)
N = 2
np.random.seed(0)
x1 = torch.Tensor(std_val * (np.random.randn(1,N) + mean_val),device=device).double()
x1.requires_grad = True
y = loss_f(x1)
#define optimizer
optimizer = torch.optim.Adam([x1], lr=4., betas=(.5,.9))
#Optimize
i = 1
last_loss = loss_f(x1).data.numpy()[0]
adj_error = last_loss
while i<=max_iter and adj_error > iter_stop:
  optimizer.zero_grad()
  loss = loss_f(x1)
  loss.backward()
  optimizer.step()
  if i % 10 == 0:  
    curr_loss = loss_f(x1).data.numpy()[0]
    adj_error = compute_loss_error(last_loss,curr_loss)
    last_loss = curr_loss
  i = i + 1
    
print("Iteration " + str(i))
print("Approx location")
print(x1.data.numpy()[0])
pprint("Approx Value")
pprint(loss_f(x1).data.numpy()[0])
print('max relative difference')
pprint(np.abs(x1.data.numpy()[0][0] - x1.data.numpy()[0][1])/np.min(np.abs(x1.data.numpy()[0])))

The Adam optimizer is much better when there are large numbers of variables (such as with Deep Learning), but this example shows that the tools within the "torch" toolkit work as we expect them to in computational environments.

## Example 4 - Similar to Example 1, but using $w^6 + x^6 + y^6 + z^6 + 4$

To demonstrate that the problem complexity matters for the gradient method of choice we use the function $w^6 + x^6 + y^6 + z^6 + 4$.  In the following few cells we use the SGD and Adam to study the general capabilities.

#### Gradient Descent

In [0]:
#define setup
from torch.autograd import Variable
offset = Variable(torch.Tensor([4.]).double(), requires_grad = False)
mean_val = 2.0
std_val = 2.0
iter_stop = 1e-7
max_iter = 10000
def compute_loss_error(l1, l2):
  return np.abs(l1 - l2)/np.max([l1,l2])

def loss_f(in_pos):
  y = in_pos.pow(6).sum() + offset
  return y
  
device = torch.device(accelerator)
np.random.seed(0)
N = 4
x1 = Variable(torch.Tensor(std_val * (np.random.randn(1,N) + mean_val),device=device).double(),requires_grad = True)
y = loss_f(x1)

pprint(y.data.numpy())
#Optimize Using Gradient Descent
learning_rate = .001
i = 1
last_loss = loss_f(x1).data.numpy()[0]
adj_error = last_loss
while i<=max_iter and adj_error > iter_stop:
  
  loss = loss_f(x1)
  loss.backward()
  with torch.no_grad():
    x1 -= learning_rate * x1.grad
    x1.grad.zero_()
  if i % 10 == 0:  
    curr_loss = loss_f(x1).data.numpy()[0]
    adj_error = compute_loss_error(last_loss,curr_loss)
    last_loss = curr_loss
  i = i + 1
    
print("Iteration " + str(i))
print("Approx location")
print(x1.data.numpy()[0])
pprint("Approx Value")
pprint(loss_f(x1).data.numpy()[0])

Notice that this blew up with the SGD optimizer, but this is fairly typical of many problems.  SGD works well when the starting point is close to the target and when the problem is quadratic, but doesn't work particularily well in many other cases.  The user can run experiments with the learning_rate to improve performance, but the improvements do not happen very fast.

#### Adam

In [0]:
#define setup
from torch.autograd import Variable
offset = Variable(torch.Tensor([4.]).double(), requires_grad = False)
mean_val = 2.
std_val = 2.
iter_stop = 1e-7
max_iter = 10000
def compute_loss_error(l1, l2):
  return np.abs(l1 - l2)/np.max([l1,l2])

def loss_f(in_pos, target):
  y = in_pos.pow(6).sum() + offset
  return y
  
device = torch.device(accelerator)
N = 4
np.random.seed(0)
x1 = Variable(torch.Tensor(std_val * (np.random.randn(1,N) + mean_val),device=device).double(),requires_grad = True)
y = loss_f(x1,0.0)
#define optimizer
optimizer = torch.optim.Adam([x1], lr=.04, betas=(.5,.9))
#Optimize
i = 1
last_loss = loss_f(x1,0.0).data.numpy()[0]
adj_error = last_loss
while i<=max_iter and adj_error > iter_stop:
  optimizer.zero_grad()
  loss = loss_f(x1, 0.0)
  loss.backward()
  optimizer.step()
  if i % 10 == 0:  
    curr_loss = loss_f(x1,0.0).data.numpy()[0]
    adj_error = compute_loss_error(last_loss,curr_loss)
    last_loss = curr_loss
  i = i + 1
    
print("Iteration " + str(i))
print("Approx location")
print(x1.data.numpy()[0])
pprint("Approx Value")
pprint(loss_f(x1,0.0).data.numpy()[0])
print('max relative difference')
pprint(np.abs(x1.data.numpy()[0][0] - x1.data.numpy()[0][1])/np.min(np.abs(x1.data.numpy()[0])))

The Adam optimizer is more robust for finding solutions rather than exploding or developing vanishing gradients.  There are always other optimizers to learn, but the use of the momentum factors improves the robutness of the convergence.  One item that is beginning to get notice is the type of solution with Adam, namely, in this simple case the maximum relative difference is much higher than with the SGD.

There is some thought that this might lead to less "generalized" solutions in some problems.

#### Adam with Multiple Optimizers

On complex problems, it has been observed that changing the betas as you get closer to a solution often leads to faster convergence.  In the next two cells, notice that even on our simple problems and using very simple criteria to change the optimizers we do achieve faster convergence.  pytorch is particularily easy to use in this regard since the compute graphs are dynamically computed.  <b>(This is a potentially strong feature for pytorch.)</b>

In [8]:
#define setup
from torch.autograd import Variable
offset = Variable(torch.Tensor([4.]).double(), requires_grad = False)
mean_val = 2.
std_val = 2.
iter_stop = 1e-7
max_iter = 10000
def compute_loss_error(l1, l2):
  return np.abs(l1 - l2)/np.max([l1,l2])

def loss_f(in_pos, target):
  y = in_pos.pow(6).sum() + offset
  return y
  
device = torch.device(accelerator)
N = 4
np.random.seed(0)
x1 = Variable(torch.Tensor(std_val * (np.random.randn(1,N) + mean_val),device=device).double(),requires_grad = True)
y = loss_f(x1,0.0)
#define optimizer
optimizer = torch.optim.Adam([x1], lr=.04, betas=(.5,.9))
optimizer2 = torch.optim.Adam([x1], lr=.04, betas=(.9,.95))
optimizer3 = torch.optim.Adam([x1], lr=.04, betas=(.9,.999))
#Optimize
i = 1
last_loss = loss_f(x1,0.0).data.numpy()[0]
adj_error = last_loss
while i<=max_iter and adj_error > iter_stop:
  if adj_error > .1:
    optimizer.zero_grad()
    loss = loss_f(x1, 0.0)
    loss.backward()
    optimizer.step()
  else:
    optimizer2.zero_grad()
    loss = loss_f(x1, 0.0)
    loss.backward()
    optimizer2.step()   
  if i % 10 == 0:  
    curr_loss = loss_f(x1,0.0).data.numpy()[0]
    adj_error = compute_loss_error(last_loss,curr_loss)
    last_loss = curr_loss
  i = i + 1
    
print("Iteration " + str(i))
print("Approx location")
print(x1.data.numpy()[0])
pprint("Approx Value")
pprint(loss_f(x1,0.0).data.numpy()[0])
print('max relative difference')
pprint(np.abs(x1.data.numpy()[0][0] - x1.data.numpy()[0][1])/np.min(np.abs(x1.data.numpy()[0])))

Iteration 381
Approx location
[-0.02688861  0.0072186  -0.0576078   0.09676172]
'Approx Value'
4.0000008576979225
max relative difference
4.724905117913807


In [0]:
#define setup
from torch.autograd import Variable
offset = Variable(torch.Tensor([4.]).double(), requires_grad = False)
mean_val = 2.
std_val = 2.
iter_stop = 1e-7
max_iter = 10000
def compute_loss_error(l1, l2):
  return np.abs(l1 - l2)/np.max([l1,l2])

def loss_f(in_pos):
  y = in_pos.pow(6).sum() + offset
  return y
  
device = torch.device(accelerator)
N = 4
np.random.seed(0)
x1 = Variable(torch.Tensor(std_val * (np.random.randn(1,N) + mean_val),device=device).double(),requires_grad = True)
y = loss_f(x1)
#define optimizer
optimizer = torch.optim.Adam([x1], lr=.04, betas=(.5,.9))
optimizer2 = torch.optim.Adam([x1], lr=.04, betas=(.9,.95))
optimizer3 = torch.optim.Adam([x1], lr=.04, betas=(.9,.999))
#Optimize
i = 1
last_loss = loss_f(x1).data.numpy()[0]
adj_error = last_loss
while i<=max_iter and adj_error > iter_stop:
  if adj_error > .1:
    optimizer.zero_grad()
    loss = loss_f(x1)
    loss.backward()
    optimizer.step()
  elif adj_error > .01:
    optimizer2.zero_grad()
    loss = loss_f(x1)
    loss.backward()
    optimizer2.step()
  else:
    optimizer3.zero_grad()
    loss = loss_f(x1)
    loss.backward()
    optimizer3.step()    
  if i % 10 == 0:  
    curr_loss = loss_f(x1).data.numpy()[0]
    adj_error = compute_loss_error(last_loss,curr_loss)
    last_loss = curr_loss
  i = i + 1
    
print("Iteration " + str(i))
print("Approx location")
print(x1.data.numpy()[0])
pprint("Approx Value")
pprint(loss_f(x1).data.numpy()[0])
print('max relative difference')
pprint(np.abs(x1.data.numpy()[0][0] - x1.data.numpy()[0][1])/np.min(np.abs(x1.data.numpy()[0])))

On this simple problem, the choice of different optimizers at different steps yielded a 28.5% performance improvement.  This is certainly something to consider when training long running NN.  It should be noted that the maximum relative difference in these tests are still fairly high compared to the initlal tests with SGD.

####  Using Multiple Optimizers with a Finish Optimizer

As discussed in [Improving Generalization Performance by Switching from Adam to SGD](https://arxiv.org/abs/1712.07628), it might be possible to improve stability by using Adam (or variants early) then switching to SGD (or variants later).  The next cell demonstrates this possibility for our simple problem.  The convergence isn't as fast as using Adam, but the resulting solution is smoother as seen in the original problem.  This is certainly worth considering.

In [0]:
#define setup
from torch.autograd import Variable
offset = Variable(torch.Tensor([4.]).double(), requires_grad = False)
mean_val = 2.
std_val = 2.
iter_stop = 1e-7
max_iter = 10000
def compute_loss_error(l1, l2):
  return np.abs(l1 - l2)/np.max([l1,l2])

def loss_f(in_pos, target):
  y = in_pos.pow(6).sum() + offset
  return y
  
device = torch.device(accelerator)
N = 4
np.random.seed(0)
x1 = Variable(torch.Tensor(std_val * (np.random.randn(1,N) + mean_val),device=device).double(),requires_grad = True)
y = loss_f(x1,0.0)
#define optimizer
optimizer = torch.optim.Adam([x1], lr=.04, betas=(.5,.9))
optimizer2 = torch.optim.SGD([x1], lr=.1)
#Optimize
i = 1
last_loss = loss_f(x1,0.0).data.numpy()[0]
adj_error = last_loss
while i<=max_iter and adj_error > iter_stop:
  if adj_error > .1:
    optimizer.zero_grad()
    loss = loss_f(x1, 0.0)
    loss.backward()
    optimizer.step()
  else:
    optimizer2.zero_grad()
    loss = loss_f(x1, 0.0)
    loss.backward()
    optimizer2.step()   
  if i % 10 == 0:  
    curr_loss = loss_f(x1,0.0).data.numpy()[0]
    adj_error = compute_loss_error(last_loss,curr_loss)
    last_loss = curr_loss
  i = i + 1
    
print("Iteration " + str(i))
print("Approx location")
print(x1.data.numpy()[0])
pprint("Approx Value")
pprint(loss_f(x1,0.0).data.numpy()[0])
print('max relative difference')
pprint(np.abs(x1.data.numpy()[0][0] - x1.data.numpy()[0][1])/np.min(np.abs(x1.data.numpy()[0])))

## Conclusion

It is comforting to see that the optimization methods in DL are variations on mathematical convex optimizations; futhermore, the entire optimization environment has a lot of flexibility for solving problems.  This is probably as good as any environment for working with larger and more complex problems since you don't have to do much of the gradient computation manually.